# Test Case LVV-t1802
This case will verify that the integration of the M2 Hexapod with SAL.
The blocks below represent the steps of the test case.

Requirements
* EFD
* Hexapod(s) powered on
* Thermal sensors attached to the six actuators of the hexapod.
* CSC running

This test will require manual verification of certain events and telemetry in the summit EFD.
Also manual verification of appropriate temperatures for each actuator.

In [27]:
from lsst.ts import salobj
from lsst.ts.idl.enums import MTHexapod
import logging
import asyncio
import os
import yaml

import numpy as np
from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd
import astropy.units as u

The following block sets the necessary environment variables for setting up the DDS/SAL communication

In [28]:
#os.environ["LSST_DDS_DOMAIN"] = 'citest'
os.environ["LSST_DDS_HISTORYSYNC"] = "10"

In [29]:
STD_WAIT = 39 # This is the amount of time to wait in between a move just to make sure that the actuators are not overheating

In [52]:
log = logging.getLogger(__name__) # This sets up the logger for the test.
#logging.setLevel(20)
logging.basicConfig(level=logging.INFO)

In [31]:
start_time = datetime.now()
test_message = "M2 Hexapod Integration Test"
#What means index=1 here?
script = salobj.Controller("Script", index=1)
await asyncio.sleep(10) #wait 10 second may help with DDS problems; closing all other kernels may help too
print(f'time to start is {datetime.now() - start_time} [s]')

time to start is 0:00:10.804689 [s]


This is how you start the remote for the CSC.

In [32]:
domain = salobj.Domain()
print (domain)

In [79]:
#index=1 is the camera hexapod!! index=2 is the M2 hexapod!
csc_index = 2
hexapod_csc = salobj.Remote(name="MTHexapod", domain=domain, index=csc_index)

print(hexapod_csc)
await asyncio.sleep(15)
#Bo:Check which hexapod we are controlling
whichHex1 = await hexapod_csc.evt_configuration.aget(timeout=10.)
print("Hexapod hex", whichHex1.MTHexapodID)

print(hexapod_csc.salinfo.index)

await hexapod_csc.start_task
#This is to bring the state machine in the right starting state
#await salobj.set_summary_state(hexapod_csc, salobj.State.ENABLED)
#await salobj.set_summary_state(hexapod_csc, salobj.State.OFFLINE)

INFO:MTHexapod:Read historical data in 0.01 sec


Hexapod hex 2
2


In [80]:
await hexapod_csc.evt_heartbeat.next(flush=True, timeout=5)

In [81]:
#may need to wait a few seconds before event shows up in EFD
await asyncio.sleep(15.)
from lsst_efd_client import EfdClient

#client = EfdClient('ncsa_teststand_efd')
client = EfdClient('summit_efd')
end = Time(datetime.now())
start = end - timedelta(hours=3)
dfe = await client.select_time_series('lsst.sal.MTHexapod.logevent_summaryState', '*', start.tai, end.tai, csc_index)

In [82]:
dfe

,MTHexapodID,priority,private_host,private_identity,private_kafkaStamp,private_origin,private_rcvStamp,private_revCode,private_seqNum,private_sndStamp,summaryState
2021-02-24 16:09:26.616000+00:00,2,0,0,MTHexapod:2,1.614183e+09,112,1.614183e+09,959a0a03,271,1.614183e+09,1
2021-02-24 16:09:35.656000+00:00,2,0,0,MTHexapod:2,1.614183e+09,112,1.614183e+09,959a0a03,272,1.614183e+09,5
2021-02-24 16:10:25.520000+00:00,2,0,0,MTHexapod:2,1.614183e+09,112,1.614183e+09,959a0a03,273,1.614183e+09,4
2021-02-24 16:17:29.945000+00:00,2,0,0,MTHexapod:2,1.614183e+09,112,1.614183e+09,959a0a03,274,1.614183e+09,5
2021-02-24 16:17:35.350000+00:00,2,0,0,MTHexapod:2,1.614183e+09,112,1.614183e+09,959a0a03,275,1.614183e+09,1
...,...,...,...,...,...,...,...,...,...,...,...
2021-02-24 18:45:41.290000+00:00,2,0,0,MTHexapod:2,1.614192e+09,112,1.614192e+09,959a0a03,4874,1.614192e+09,5
2021-02-24 18:45:41.330000+00:00,2,0,0,MTHexapod:2,1.614192e+09,112,1.614192e+09,959a0a03,4875,1.614192e+09,3
2021-02-24 18:45:41.380000+00:00,2,0,0,MTHexapod:2,1.614192e+09,112,1.614192e+09,959a0a03,4876,1.614192e+09,5
2021-02-24 18:45:52.722000+00:00,2,0,0,MTHexapod:2,1.614192e+09,112,1.614192e+09,959a0a03,4877,1.614192e+09,1


In [83]:
def enabled_substate_callback(evt):
    """Print the enabled substate when event is received."""
    print(MTHexapod.EnabledSubstate(evt.enabledSubstate))

In [84]:
def in_position_callback(evt):
    """Print the in position event when it is received."""
    print(evt.in_position)

This next block will make sure that the CSC is ready for the test

In [85]:
connected = await hexapod_csc.evt_connected.aget() # This is how you get an event/telemetry with await remote.type_name.aget() type being [evt,tel] and name being the name of the topic
commandable = connected.command
telemetry_working = connected.telemetry

if not connected or not telemetry_working:
    raise Exception("Hexapod not connected or telemetry not being received.")

    #This is only for the real system. The simulater only has the AVAILABLE offline state 
 
controller_state = await hexapod_csc.evt_controllerState.aget()
state = controller_state.controllerState
offline_substate = controller_state.offlineSubstate
print(state)
'''  
if not state == salobj.State.OFFLINE and not offline_substate == MTHexapod.OfflineSubstate.AVAILABLE:
    raise Exception("Controller must be changed to Available Offline Substate.")
'''
commandable_by_dds = await hexapod_csc.evt_commandableByDDS.aget()
dds_state = commandable_by_dds.state
if not dds_state:
    raise Exception("Controller must in CommandableByDDS state.")


2


The next four blocks bring the CSC to the Enabled state.

In [ ]:
await hexapod_csc.cmd_enterControl.set_start() # Send the enterControl command notice the casing and the set_start method

In [112]:
await hexapod_csc.cmd_start.set_start() 

EnabledSubstate.STATIONARY


In [113]:
await hexapod_csc.cmd_enable.set_start()

EnabledSubstate.STATIONARY


In [115]:
await hexapod_csc.cmd_clearError.set_start() # This clears the error

EnabledSubstate.STATIONARY


AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=158026926, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.ENABLED: 2> instead of <State.FAULT: 3>')

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_PO

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [47]:
# Step skiped see deviation LVV-T1802 Test Step 5                                  
#hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
#await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900, u=0.12,v=0,w=0,sync=True)



In [87]:
await hexapod_csc.cmd_stop.set_start()

EnabledSubstate.STATIONARY


In [101]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


In [19]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 7")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

hexapod_csc.evt_controllerState.callback = enabled_substate_callback                                
await hexapod_csc.cmd_move.set_start(x=500,y=-500,z=200, u=0.01,v=-0.015,w=0,sync=True)
await asyncio.sleep(15.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 7")


hex position
      0.05        0.30       -0.36       -0.00        0.00        0.00  

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
    500.24     -500.36      199.90        0.01       -0.02        0.00  



In [20]:
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [21]:
# thermal sensors

This block moves the hexapod to a large position but stops it after waiting 3 seconds

In [53]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 11")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

await hexapod_csc.cmd_move.set_start(x=0,y=0,z=5000,u=0,v=0,w=0,sync=True)
await asyncio.sleep(3)
await hexapod_csc.cmd_stop.set_start()

await asyncio.sleep(15.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


#cs = await hexapod_csc.evt_controllerState.aget()
#print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
#if cs.enabledSubstate != MTHexapod.EnabledSubstate.CONTROLLED_STOPPING:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.CONTROLLED_STOPPING}. It is {MTHexapod.EnabledSubstate(cs.enabledSubstate)}")

# cs = await hexapod_csc.evt_controllerSubstate.aget()
# print(MTHexapod.EnabledSubstate(cs.enabledSubstate))
# if cs.enabledSubstate != MTHexapod.EnabledSubstate.STATIONARY:
#     raise Exception(f"Controller's substate is not {MTHexapod.EnabledSubstate.STATIONARY}. It is {MtHexapod.EnabledSubstate(cs.enabledSubstate)}")

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 11")


INFO:Script:START- M2 Hexapod Integration Test -- LVV-T1802 Test Step 11


hex position
      0.16        0.45       -0.34       -0.00        0.00        0.00  

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- M2 Hexapod Integration Test -- LVV-T1802 Test Step 11


hex position
     -9.43       17.09     1241.17        0.00       -0.00        0.00  

EnabledSubstate.STATIONARY


In [54]:
#Step 15
await asyncio.sleep(STD_WAIT)

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [103]:
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=0, u=0,v=0,w=0,sync=True)

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


In [75]:
await asyncio.sleep(STD_WAIT)

EnabledSubstate.STATIONARY


This block moves the hexapod via the lookup table:

In [117]:
#This command is obsolete
#await hexapod_csc.cmd_moveLUT.set_start(az=180,elev=60, temp=10)

script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 17")
await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')



#Switch CompensationMode on
await hexapod_csc.cmd_setCompensationMode.set_start(enable=True)

lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)


#move the hexapod Not allowed here! First leave compensation mode than move to new postion!!! Otherwise the you hexpod gets stuck in compensation  mode
#await hexapod_csc.cmd_move.set_start(x=0,y=0,z=800,u=0,v=0,w=0,sync=True)

#await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

#Switch CompensationMode off 
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)
lutMode = await hexapod_csc.evt_compensationMode.aget(timeout=10)
print("compsensation mode enabled?",lutMode.enabled)

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 17")

INFO:Script:START- M2 Hexapod Integration Test -- LVV-T1802 Test Step 17


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
     -0.09        0.00      799.94       -0.00       -0.00        0.00  

compsensation mode enabled? True
hex position
     -0.09        0.00      799.94       -0.00       -0.00        0.00  

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubsta

INFO:Script:STOP- M2 Hexapod Integration Test -- LVV-T1802 Test Step 17


hex position
     -0.09        0.00      799.94       -0.00       -0.00        0.00  

compsensation mode enabled? False
EnabledSubstate.STATIONARY


In [116]:
await hexapod_csc.cmd_setCompensationMode.set_start(enable=False)


EnabledSubstate.MOVING_POINT_TO_POINT


EnabledSubstate.STATIONARY


In [118]:
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

hex position
     -0.08        0.01      799.94       -0.00       -0.00        0.00  



In [119]:
posC = await hexapod_csc.evt_compensatedPosition.aget(timeout=10.)
print('Compensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')
posU = await hexapod_csc.evt_uncompensatedPosition.aget(timeout=10.)
print('Uncompensated position')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posU, i), end= ' ')
print('\n')

Compensated position
      0.00        0.00      800.00        0.00        0.00        0.00  

Uncompensated position
      0.00        0.00      800.00        0.00        0.00        0.00  



Analysis of the LUT correction

In [120]:
LUTfile = '%s/notebooks/ts_config_mttcs/MTHexapod/v1/default.yaml'%(os.environ["HOME"])
print(LUTfile)
with open(LUTfile, 'r') as stream:
    aa = yaml.safe_load(stream)
if hexapod_csc.salinfo.index == 1:
    elevCoeff = aa['camera_config']['elevation_coeffs']
    tCoeff = aa['camera_config']['temperature_coeffs']
elif hexapod_csc.salinfo.index == 2:
    elevCoeff = aa['m2_config']['elevation_coeffs']
    tCoeff = aa['m2_config']['temperature_coeffs']

/home/hdrass/notebooks/ts_config_mttcs/MTHexapod/v1/default.yaml


In [121]:
pred = []
elev=89.999
print('Predicted LUT compensation:')
for i in range(6):
    coeff = elevCoeff[i] #starts with C0
    mypoly = np.poly1d(coeff[::-1]) #poly1d needs C5 first
    pred.append(mypoly(elev))
    print('%10.2f '%pred[i], end= ' ')
print('\n')
print('Actual LUT compensation:')
for i in 'xyzuvw':
    print('%10.2f '%getattr(posC, i), end= ' ')
print('\n')

Predicted LUT compensation:
     -1.84     -572.57      304.70       -0.01        0.00        0.00  

Actual LUT compensation:
      0.00        0.00      800.00        0.00        0.00        0.00  



In [95]:
#Step 22
await asyncio.sleep(STD_WAIT)
# thermal sensors

# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [100]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 24")

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')


await hexapod_csc.cmd_move.set_start(x=500,y=800,z=200,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

await hexapod_csc.cmd_offset.set_start(x=0,y=0,z=500,u=0,v=0,w=0,sync=True)
await asyncio.sleep(10.)

pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 24")


# cs = await MThexapod_csc.evt_controllerState.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.MOVE_POINT_TO_POINT}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# cs = await MThexapod_csc.evt_controllerSubstate.aget()
# if cs.enabledSubstate != MThexapod.SetEnabledSubstateParam.STATIONARY:
#     raise Exception(f"Controller's substate is not {MThexapod.SetEnabledSubstateParam.STATIONARY}. It is {MThexapod.SetEnabledSubstateParam(cs.enabledSubstate)}")
# in_position = await MThexapod_csc.evt_inPosition.aget(timeout=10)
# if in_position.inPosition is False:
#     raise Exception("Controller not in position.")

INFO:Script:START- M2 Hexapod Integration Test -- LVV-T1802 Test Step 24


hex position
     -0.03       -0.21       -0.22        0.00        0.00        0.00  

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
    500.05      800.33      199.87       -0.00       -0.00        0.00  

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY


INFO:Script:STOP- M2 Hexapod Integration Test -- LVV-T1802 Test Step 24


hex position
    500.37      799.63      699.90       -0.00       -0.00       -0.00  



In [ ]:
#Step 27
await asyncio.sleep(STD_WAIT)

In [104]:
script.log.info(f"START- {test_message} -- LVV-T1802 Test Step 29")
#Step 29
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')

#Step 30 -- set a new Pivot point
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=0)
await asyncio.sleep(10.)
#Step 31
await hexapod_csc.cmd_move.set_start(x=2000,y=-3500,z=200,u=0.01,v=-0.05,w=0.002,sync=True)
await asyncio.sleep(10.)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
await asyncio.sleep(10.)
#Set the pivot point back to the original value (0.703m in um)
await hexapod_csc.cmd_setPivot.set_start(x=0,y=0,z=703000)
pos = await hexapod_csc.tel_application.aget(timeout=10.)
print('hex position')
for i in range(6):
    print('%10.2f '%pos.position[i], end= ' ')
print('\n')
script.log.info(f"STOP- {test_message} -- LVV-T1802 Test Step 29")

INFO:Script:START- M2 Hexapod Integration Test -- LVV-T1802 Test Step 29


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
   2000.33    -3500.21      199.97        0.01       -0.05        0.00  

EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
hex position
   2000.33    -3500.18      199.96        0.01       -0.05        0.00  



INFO:Script:STOP- M2 Hexapod Integration Test -- LVV-T1802 Test Step 29


hex position
   2000.33    -3500.18      199.96        0.01       -0.05        0.00  



# thermal sensors
Check the chronograph manually for the temperature sensors to be below 19C, if not wait until they are all below 19C. Enter in the data at the 39 second mark

|Actuator 1 (C)|Actuator 2 (C) | Actuator 3 (C) | Actuator 4 (C) | Actuator 5 (C) | Actuator 6 (C)|
|--------------|---------------|----------------|----------------|----------------|---------------|
| 0 | 0 | 0 | 0 | 0 | 0 |

In [122]:


script.log.info(f"START- {test_message} -- LVV-T1802 Endurance Test")
for i in range(100):
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=-6700,y=0,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)

    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0,v=-0.12,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)
    await hexapod_csc.cmd_move.set_start(x=0,y=-6700,z=-5900,u=-0.12,v=0,w=0,sync=True)
    await asyncio.sleep(STD_WAIT)


script.log.info(f"STOPP- {test_message} -- LVV-T1802 Endurance Test ")

INFO:Script:START- M2 Hexapod Integration Test -- LVV-T1802 Endurance Test


EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.MOVING_POINT_TO_POINT
EnabledSubstate.STATIONARY
EnabledSubstate.STATIONAR

AckError: msg='Command failed', ackcmd=(ackcmd private_seqNum=1008276314, ack=<SalRetCode.CMD_FAILED: -302>, error=1, result='Failed: Rejected: initial state is <State.FAULT: 3> instead of <State.ENABLED: 2>')

In [ ]:
await salobj.set_summary_state(hexapod_csc, salobj.State.STANDBY) # Transition the CSC to Standby state

In [ ]:
await domain.close() # Close the remote connection